In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import clear_output
import os
import gzip
import shutil
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from natsort import natsort_keygen
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mattroth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Function to download genres dataset from UCSD source
def download_data(filename, new_filename):
   
    """
    Downloads raw goodreads data from UCSD 
    ______

    filename(str): name of the file as hosted by UCSD. e.g. "goodreads_books.json.gz"
    new_filename(str): how you want to save the file locally. e.g. "goodreads_books.json"
    """

    wd = os.getcwd()
    url = "https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/" + filename
        
    def download_by_name(fname, local_filename):

        with requests.get(url, stream=True) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
            
    output_path = os.path.join(wd, filename)
    download_by_name(filename, output_path)

    # Unzip and convert to json
    with gzip.open(filename, 'rb') as f_in:
        with open(new_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [3]:
# Read in select fields from long version of books table
wd = os.getcwd()
col_names = ["book_id","title","description","language_code","ratings_count"]
json_data = []
download_data("goodreads_books.json.gz", "goodreads_books.json")
file_name = wd + "/goodreads_books.json"

with open(file_name, encoding='UTF-8') as f:
    i = 0
    for line in f:
        doc = json.loads(line)
        lst = [doc[col_names[i]] for i in range(len(col_names))]
        json_data.append(lst)
        if i % 50000 == 0:
            clear_output(wait=True)
            print(f"Processing row: {i:,d}")
        i += 1
    print(f"Completed! Processed {i:,d} rows")

df_book_descriptions = pd.DataFrame(data=json_data, columns=col_names)

# Remove files
os.remove(wd + "/goodreads_books.json")
os.remove(wd + "/goodreads_books.json.gz")

Processing row: 2,350,000
Completed! Processed 2,360,655 rows


In [4]:
df_book_descriptions.head()

book_id                                              title  \
0  5333265                        W.C. Fields: A Life on Film   
1  1333909                                        Good Harbor   
2  7327624  The Unschooled Wizard (Sun Wolf and Starhawk, ...   
3  6066819                               Best Friends Forever   
4   287140  Runic Astrology: Starcraft and Timekeeping in ...   

                                         description language_code  \
0                                                                    
1  Anita Diamant's international bestseller "The ...                 
2  Omnibus book club edition containing the Ladie...           eng   
3  Addie Downs and Valerie Adler were eight when ...           eng   
4                                                                    

  ratings_count  
0             3  
1            10  
2           140  
3         51184  
4            15

In [5]:
# Get rid of null descriptions
df_book_descriptions = df_book_descriptions[df_book_descriptions["description"] != ""]

# Filter out books with less than 150 ratings
df_book_descriptions['ratings_count'] = pd.to_numeric(df_book_descriptions["ratings_count"], errors='coerce').astype('Int32')
df_book_descriptions = df_book_descriptions[df_book_descriptions["ratings_count"] > 150]

# Keep only english or null language books
df_book_descriptions = df_book_descriptions[df_book_descriptions["language_code"].isin(["","eng","en-US","en-GB","en-CA"])]

# Cut down to just id, title, and description
df_book_descriptions = df_book_descriptions[["book_id", "title", "description"]]
df_book_descriptions.head()

book_id                                              title  \
3   6066819                               Best Friends Forever   
9    287149                               The Devil's Notebook   
10  6066814          Crowner Royal (Crowner John Mystery, #13)   
15    89375  90 Minutes in Heaven: A True Story of Death an...   
16    89376                                             Heaven   

                                          description  
3   Addie Downs and Valerie Adler were eight when ...  
9   Wisdom, humor, and dark observations by the fo...  
10  London, 1196. At the command of Richard the Li...  
15  As he is driving home from a minister's confer...  
16  What is Heaven really going to be like? What w...

In [6]:
# Initialize regex tokenizer
tokenizer = RegexpTokenizer(
    "(?:(?<=\s)|(?<=^)|(?<=[>\"]))[a-z-']+(?:(?=\s)|(?=\:\s)|(?=$)|(?=[.!,;\"]))"
)

# Lemmatize description text
lem = WordNetLemmatizer()
def lemmatize_text(text):
    return [lem.lemmatize(w) for w in tokenizer.tokenize(text)]

# Add lemmatized version of description to df_book_descriptions
df_book_descriptions['description_lemmatized'] = df_book_descriptions["description"].apply(lambda x: lemmatize_text(x))
df_book_descriptions["description_lemmatized"] = df_book_descriptions["description_lemmatized"].apply(lambda x: " ".join(x))

# Create custom stop words set with units and others
custom_stop_words = text.ENGLISH_STOP_WORDS.union([
    "ha","wa","-","--","u","life","he's","she's","didn't","doesn't","don't","can't","le","who's","isn't","wasn't","they're",
    "man","woman","mom","dad","mother","father","brother","sister","son","daughter","girl","boy",
    "character","story","novel","book","author","edition","new","reader",
    'want','thing','like','make','need',"it's","know","come",
    'work','writer','volume','published','series','tale','original',
    'wanted', 'knew', 'met', 'love', 'got', 'thought', 'just', 'away', 'did',
    'la', 'en', 'y', 'que', 'el', 'los', 'su', 'una', 'del', 'se',
    'e', 'o', 'na', 'da', 'di', 'se', 'v', 'la', 'je', 'si', 's,','t',
    's', 'et', 'il', 'sa', 'ne', 'du', 'che', 'qui', 'une', 'ce',
    'mn', 'fy', 'w', "'n", 'l', 'm', 'min', 'lktb', 'lty',
    'und', 'der', 'er', 'sie', 'den', 'von', 'zu',
    'por', 'para', 'ma', 'como', 'al', 'sus', 'lo', 'vida', 'esta', 'historia',
    ","
    ])

# Encode data with TF-IDF
tfidf = TfidfVectorizer(lowercase=True,
                        stop_words=list(custom_stop_words),
                        max_df=0.75, # ignore words occuring in more than this % of docs
                        min_df=0.002, # ignore words occuring in less than this % of docs
                        max_features=10000,
                        tokenizer=tokenizer.tokenize)
vectorized_descriptions = tfidf.fit_transform(df_book_descriptions["description_lemmatized"])

# Save vocab
vocab = tfidf.get_feature_names_out()

/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [7]:
# Instantiate LDA
n_topics = 25
lda = LDA(n_components=n_topics, n_jobs=-1)

# Run on vectorized_descriptions
X_topics = lda.fit_transform(vectorized_descriptions)

In [8]:
# Get topic distribution
topic_words = lda.components_

# Retrieve topics
genre_strings = []
for i, topic_dist in enumerate(topic_words):
    
    # Zip vocab and topic distribution
    zipped_10 = zip(vocab, topic_dist)    

    # Sort and get out top 1 words for each topic for dict
    top_terms_key_10 = sorted(zipped_10, key=lambda t: t[1], reverse=True)[0:10]
    top_terms_list_10 = list(dict(top_terms_key_10).keys())

    # Print genre top words
    genre_string = f"Genre {i + 1}: {top_terms_list_10}"
    genre_strings.append(genre_string)
    print(genre_string)
    

Genre 1: ['family', 'young', 'house', 'home', 'island', 'village', 'secret', 'world', 'old', 'beautiful']
Genre 2: ['sexual', 'sex', 'contains', 'romance', 'content', 'language', 'sexy', 'read', 'hot', 'explicit']
Genre 3: ['collection', 'comic', 'short', 'fan', 'feature', 'bestselling', 'adventure', 'available', 'classic', 'includes']
Genre 4: ['guy', 'friend', 'sexy', 'job', 'best', 'time', 'way', 'night', 'bad', 'hot']
Genre 5: ['history', 'war', 'political', 'account', 'world', 'year', 'country', 'family', 'century', 'people']
Genre 6: ['secret', 'thriller', 'mystery', 'truth', 'murder', 'world', 'spy', 'conspiracy', 'agent', 'mysterious']
Genre 7: ['fairy', 'magic', 'adventure', 'world', 'magical', 'die', 'fantasy', 'journey', 'young', 'land']
Genre 8: ['guide', 'people', 'way', 'help', 'practical', 'world', 'experience', 'offer', 'question', 'business']
Genre 9: ['murder', 'crime', 'prison', 'trial', 'case', 'lawyer', 'sin', 'law', 'justice', 'year']
Genre 10: ['killer', 'murder'

In [9]:
# Extract top topic for all descriptions
inferred_genres = lda.transform(vectorized_descriptions)

topics = []
for i in range(inferred_genres.shape[0]):

    # Get most likely topic for each descriptions
    topic = inferred_genres[i].argmax() + 1
    topics.append(topic)

# Add columns to df_book_descriptions
df_book_descriptions["main_genre"] = topics
for i in range(inferred_genres.shape[1]):
    df_book_descriptions[f"Genre_{i+1}"] = inferred_genres[:,i]

# Inspect df_book_descriptions with topic
df_book_descriptions.head()    

book_id                                              title  \
3   6066819                               Best Friends Forever   
9    287149                               The Devil's Notebook   
10  6066814          Crowner Royal (Crowner John Mystery, #13)   
15    89375  90 Minutes in Heaven: A True Story of Death an...   
16    89376                                             Heaven   

                                          description  \
3   Addie Downs and Valerie Adler were eight when ...   
9   Wisdom, humor, and dark observations by the fo...   
10  London, 1196. At the command of Richard the Li...   
15  As he is driving home from a minister's confer...   
16  What is Heaven really going to be like? What w...   

                               description_lemmatized  main_genre   Genre_1  \
3   and were eight when they first met and decided...          14  0.006364   
9   humor and dark observation by the founder of t...          15  0.009747   
10  the command of the de ha left his beloved for ...           6  0.006272   
15  he is driving home from a minister's conferenc...          20  0.006648   
16  is really going to be will we look will we get...           8  0.004567   

     Genre_2   Genre_3   Genre_4   Genre_5  ...  Genre_16  Genre_17  Genre_18  \
3   0.006364  0.006364  0.006364  0.006364  ...  0.006364  0.006364  0.006364   
9   0.009747  0.009747  0.009747  0.009747  ...  0.009747  0.009747  0.009747   
10  0.006272  0.006272  0.006272  0.006272  ...  0.006272  0.006272  0.006272   
15  0.006648  0.006648  0.006648  0.006648  ...  0.006648  0.006648  0.006648   
16  0.004567  0.004567  0.004567  0.004567  ...  0.283212  0.004567  0.004567   

    Genre_19  Genre_20  Genre_21  Genre_22  Genre_23  Genre_24  Genre_25  
3   0.006364  0.006364  0.006364  0.006364  0.006364  0.006364  0.006364  
9   0.009747  0.009747  0.009747  0.009747  0.009747  0.009747  0.009747  
10  0.006272  0.006272  0.006272  0.006272  0.006272  0.006272  0.006272  
15  0.006648  0.840454  0.006648  0.006648  0.006648  0.006648  0.006648  
16  0.004567  0.004567  0.004567  0.004567  0.004567  0.004567  0.004567  

[5 rows x 30 columns]

In [10]:
# Save as csv
df_book_descriptions_short = df_book_descriptions.drop(["description", "description_lemmatized"], axis=1)
pd.DataFrame(genre_strings,columns=["genre_string"]).to_csv(wd + "/data/inferred_genre_top_words.csv", index=False)
df_book_descriptions_short.to_csv(wd + "/data/inferred_genres.csv", index=False)

In [11]:
# Function to plot word importances to topics
def plot_top_words(model, feature_names):
    
    fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(12, 12), sharex=True)
    axes = axes.flatten()
    
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -10 - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, alpha=0.6)
        ax.set_title(f"Genre {topic_idx +1}")
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major")
        for i in "top right".split():
            ax.spines[i].set_visible(False)
        fig.suptitle("Top Words by LDA Inferred Genres", fontsize=22, y=1.01)
        if topic_idx >= 15:
            ax.set_xlabel("Weight")

    plt.tight_layout()
    plt.show()

plot_top_words(lda, vocab)

In [12]:
# Plot topics by number of books where each topic is the main topic
topic_count = df_book_descriptions.groupby("main_genre")["description"].count().reset_index()\
    .sort_values(by="description", ascending=False)
topic_count["topic_name"] = [f"Genre {t}" for t in topic_count["main_genre"]]

fig, ax = plt.subplots(figsize=(10,8))
sns.barplot(
    data=topic_count,
    y="topic_name",
    x="description",
    color="tab:orange",
    alpha=0.6
)
plt.ylabel(None)
plt.xlabel("Count of Books")
plt.title("Count of Books by Main Genre")
plt.show()